# Tune Model Hyperparameters


You may have noticed that AutoGluon tries multiple models from decision trees to neural network during fit. If you are familiar with these models, you can specify what models AutoGluon should try, and model hyperparameters AutoGluon should search. 

Let's first figure what models AutoGluon trained, with their performances on the test set through the {class}`autogluon.tabular.TabularPredictor.leaderboard` method.


In [1]:
#@title Install autogluon
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 14.8 MB/s 
     |████████████████████████████████| 203 kB 47.1 MB/s 
     |████████████████████████████████| 48 kB 6.4 MB/s 
     |████████████████████████████████| 272 kB 65.3 MB/s 
     |████████████████████████████████| 63 kB 2.7 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
     |████████████████████████████████| 61 kB 245 kB/s 
     |████████████████████████████████| 132 kB 52.7 MB/s 
     |████████████████████████████████| 802 kB 56.1 MB/s 
     |████████████████████████████████| 38.1 MB 48.9 MB/s 
     |████████████████████████████████| 1.0 MB 49.0 MB/s 
     |████████████████████████████████| 54.5 MB 104 kB/s 
     |████████████████████████████████| 1.6 MB 50.5 MB/s 
     |████████████████████████████████| 281 kB 55.5 MB/s 
     |████████████████████████████████| 13.5 MB 50.5 MB/s 
     |███████████████████████████████

In [1]:
#@title Load the knot theory data and fit
from autogluon.tabular import TabularDataset, TabularPredictor

url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(url+'train.csv')
test_data = TabularDataset(url+'test.csv')
label = 'signature'

predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20220710_164621/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220710_164621/"
AutoGluon Version:  0.5.0
Python Version:     3.9.12
Operating System:   Linux
Train Data Rows:    10000
Train Data Columns: 18
Label Column: signature
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 13) unique label values:  [-2, 0, 2, -8, 4, -4, -6, 8, 6, 10]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9984
Train Data Class Count: 9
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Ava

In [3]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.964965,0.052081,17.529240,0.000582,0.624790,2,True,14
1,XGBoost,0.956957,0.026818,6.198938,0.026818,6.198938,1,True,11
2,CatBoost,0.955956,0.007128,18.482011,0.007128,18.482011,1,True,8
3,LightGBM,0.955956,0.024875,4.012159,0.024875,4.012159,1,True,5
4,LightGBMLarge,0.949950,0.036438,9.632664,0.036438,9.632664,1,True,13
5,RandomForestEntr,0.949950,0.119745,1.264224,0.119745,1.264224,1,True,7
6,ExtraTreesGini,0.946947,0.119009,1.159015,0.119009,1.159015,1,True,9
7,LightGBMXT,0.945946,0.028238,5.506335,0.028238,5.506335,1,True,4
8,NeuralNetFastAI,0.943944,0.024681,10.705512,0.024681,10.705512,1,True,3
9,RandomForestGini,0.942943,0.118767,1.192410,0.118767,1.192410,1,True,6


Each row contains a model, with its validation score (`score_val`), prediction time on the validation set (`pred_time_val`), and training time (`fit_time`). The score is normalized from the validation metric (see {doc}`evaluations` for more details), the higher the better. 

From this leaderboard, you can get a sense about each model's accuracy versus computational cost on your data. For example, the model with highest score is `WeightedEnsemble_L2`, which a weighted combination of multiple models. `XGBoost` performs well as long as other tree models. Neural networks such as `NeuralNetFastAI` and `NeuralNetTorch` report medium scores but are more expensive to train.

```{hint}
Beyound on the validation set, you can evaluate models on a test dataset by `predictor.leaderboard(test_data, ...)`. But if you are doing it, you may use `test_data` as validation data to tune your models.
```

Next let's manually construct the search space. Through the `hyperparameters` argument, we specify the space for each type of the model. For each type, we can either specify a hyperparameter to a particular value, or a set of candidates. Then we specify how to search a good hyperparameter configuration through the `hyperparameter_tune_kwargs` argument.



In [6]:
# from autogluon.core import space

nn_options = { # for neural network
  'num_epochs': 10, # controls the training time of neural networks
  # search learning rate on log-scale
  'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),
  # activation functions, the first entry is the default
  'activation': space.Categorical('relu', 'softrelu', 'tanh'),
  # dropout probability
  'dropout_prob': space.Real(0.0, 0.5, default=0.1)
}

gbm_options = {  # for lightGBM
  # controls the training time of GBM models
  'num_boost_round': 100,  
  # number of leaves, controls the model complexity 
  'num_leaves': space.Int(lower=26, upper=66, default=36), 
}

# Only try two models
hyperparameters = {'GBM': gbm_options, 'NN_TORCH': nn_options }

# How to search in the hyperparameter space.
hyperparameter_tune_kwargs = {
    # Try at most 5 hyperparameter configurations for each type of model
    'num_trials': 5,
    # Search on local machine, you can search on a cluster
    'scheduler': 'local',
    # Automatically choose the hyperparameter optimization method
    'searcher': 'auto'
}

predictor = TabularPredictor(label=label).fit(
    train_data, hyperparameters=hyperparameters, 
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20220710_164913/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220710_164913/"
AutoGluon Version:  0.5.0
Python Version:     3.9.12
Operating System:   Linux
Train Data Rows:    10000
Train Data Columns: 18
Label Column: signature
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 13) unique label values:  [-2, 0, 2, -8, 4, -4, -6, 8, 6, 10]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9984
Train Data Class Count: 9
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Ava

  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	0.9544	 = Validation score   (accuracy)
	1.14s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.9549	 = Validation score   (accuracy)
	0.99s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.9554	 = Validation score   (accuracy)
	1.73s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.9204	 = Validation score   (accuracy)
	0.92s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.9509	 = Validation score   (accuracy)
	1.25s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ...


  0%|          | 0/5 [00:00<?, ?it/s]

Fitted model: NeuralNetTorch/T1 ...
	0.8838	 = Validation score   (accuracy)
	2.58s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/T2 ...
	0.9194	 = Validation score   (accuracy)
	5.35s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/T3 ...
	0.8042	 = Validation score   (accuracy)
	4.41s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/T4 ...
	0.9269	 = Validation score   (accuracy)
	4.39s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: NeuralNetTorch/T5 ...
	0.8312	 = Validation score   (accuracy)
	5.27s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.9604	 = Validation score   (accuracy)
	0.71s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 30.28s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-202207

Use `leaderboard` again, you will see the model performance of each trial.  

In [5]:
predictor.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.960441,0.218941,20.934792,0.000673,0.705059,2,True,11
1,LightGBM/T3,0.955433,0.036001,1.720419,0.036001,1.720419,1,True,3
2,LightGBM/T2,0.954932,0.019137,0.952366,0.019137,0.952366,1,True,2
3,LightGBM/T1,0.954432,0.026154,1.086735,0.026154,1.086735,1,True,1
4,LightGBM/T5,0.950926,0.034309,1.309738,0.034309,1.309738,1,True,5
5,NeuralNetTorch/T4,0.926890,0.029172,4.471529,0.029172,4.471529,1,True,9
6,LightGBM/T4,0.920381,0.035940,0.947361,0.035940,0.947361,1,True,4
7,NeuralNetTorch/T2,0.919379,0.015999,5.255046,0.015999,5.255046,1,True,7
8,NeuralNetTorch/T1,0.883826,0.017418,2.737767,0.017418,2.737767,1,True,6
9,NeuralNetTorch/T5,0.831247,0.026955,5.371318,0.026955,5.371318,1,True,10


Note that constructing a good search space requires domain knowledge. The space should be large enough to cover good configurations, but should not be too large to search. You can start with AutoGluon's default hyperparameters, and interatively refine the search space.


```{seealso}
Know all model type (LINK) and their hyperparameters (LINK). 

Know all search algorithms (LINK)

You can also add your customized models.

External docs about HPO.
```

(TODO, how to reuse models traind before when you are iteratively refining the search space).